# PyCitySchools Challenge

### Importing modules needed and setting up file_to_open paths

In [14]:
import pandas as pd
import numpy as np
import os

school_data_to_load = os.path.join("Resources", "schools_complete.csv")
student_data_to_load = os.path.join("Resources", "students_complete.csv")

school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

### Correcting the students' names so there are no professional prefixes or suffixes.

In [15]:
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

### Replace the reading and math scores for ninth graders at Thomas High School with NaN.

In [16]:
student_data_df.loc[(student_data_df["school_name"] == "Thomas High School") & (student_data_df["grade"] == "9th"), ["reading_score", "math_score"]] = np.nan
student_data_df.tail(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
39160,39160,Katie Weaver,F,11th,Thomas High School,89.0,86.0
39161,39161,April Reyes,F,10th,Thomas High School,70.0,84.0
39162,39162,Derek Weeks,M,12th,Thomas High School,94.0,77.0
39163,39163,John Reese,M,11th,Thomas High School,90.0,75.0
39164,39164,Joseph Anthony,M,9th,Thomas High School,NaN,NaN
39165,39165,Donna Howard,F,12th,Thomas High School,99.0,90.0
39166,39166,Dawn Bell,F,10th,Thomas High School,95.0,70.0
39167,39167,Rebecca Tanner,F,9th,Thomas High School,NaN,NaN
39168,39168,Desiree Kidd,F,10th,Thomas High School,99.0,90.0
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95.0,75.0


### Merge the clean student data with the school dataset. The column order for all the DataFrames and number formatting should be the same as what was covered in this module.

In [17]:
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])

student_count = school_data_complete_df["Student ID"].count()
school_count = len(school_data_complete_df["school_name"].unique())
total_budget = school_data_complete_df["budget"].unique().sum()

### Recreate the district and school summary DataFrames

In [18]:
#creating a function fo calculating percentages
def percentage_calc(count, total_count):
    return count/float(total_count)* 100

average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]

passing_reading_count = passing_reading["student_name"].count()
passing_math_count = passing_math["student_name"].count()

passing_reading_percentage = percentage_calc(passing_reading_count, student_count)
passing_math_percentage = percentage_calc(passing_math_count, student_count)

passing_both = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70) & (school_data_complete_df["math_score"] >= 70)]
passing_both_count = passing_both["student_name"].count()
passing_both_percentage = percentage_calc(passing_both_count, student_count)

#### District Summary

In [19]:
district_summary_df = pd.DataFrame([{"Total Schools": school_count, "Total Students": student_count, "Total Budget": total_budget, "Average Reading Score": average_reading_score, "Average Math Score": average_math_score, "% Passing Reading": passing_reading_percentage, "% Passing Math": passing_math_percentage, "% Overall Passing": passing_both_percentage}])

district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.2f}%".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.2f}%".format)

new_column_order = ["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]
district_summary_df = district_summary_df[new_column_order]
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.9,81.9,73.88%,84.65%,64.09%


With the 9th grade scores of THS eliminated, the overall passing percentage dropped from 64.09% to 65.17%. This also means that the average math score droped from 79 to 78.9, and the % of students passing math also dropeed from 74.98% to 85.81%. That said, while % of students passing reading dropped from 85.81% to 84.65%, the average reading score remained the same at 81.9.

#### School Summary

In [24]:
per_school_types = school_data_df.set_index(["school_name"])["type"]
per_school_budget = school_data_df.set_index(["school_name"])["budget"]
per_school_counts = school_data_complete_df["school_name"].value_counts()
per_school_averages = school_data_complete_df.groupby(["school_name"]).mean()

per_school_math_average = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading_average = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

per_school_capita = per_school_budget / per_school_counts
per_school_types_df = pd.DataFrame(per_school_types)

per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

per_school_passing_math_count = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading_count = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

per_school_passing_math_per = per_school_passing_math_count / per_school_counts * 100
per_school_passing_reading_per = per_school_passing_reading_count / per_school_counts* 100

per_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]
per_passing_math_reading_count = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]
per_overall_passing_percentage = per_passing_math_reading_count / per_school_counts * 100

per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math_average,
           "Average Reading Score": per_school_reading_average,
           "% Passing Math": per_school_passing_math_per,
           "% Passing Reading": per_school_passing_reading_per,
           "% Overall Passing": per_overall_passing_percentage})

##### This section is commentated out because if formatting is applied to this Dataframe, there will be a KeyError in calling these variables again.


In [25]:
# per_school_summary_df["Total Students"] = per_school_summary_df["Total Students"].map("{:,}".format)
# per_school_summary_df["Total Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
# per_school_summary_df["Average Reading Score"] = per_school_summary_df["Average Reading Score"].map("{:.1f}".format)
# per_school_summary_df["Average Math Score"] = per_school_summary_df["Average Math Score"].map("{:.1f}".format)
# per_school_summary_df["% Passing Reading"] = per_school_summary_df["% Passing Reading"].map("{:.2f}%".format)
# per_school_summary_df["% Passing Math"] = per_school_summary_df["% Passing Math"].map("{:.2f}%".format)
# per_school_summary_df["% Overall Passing"] = per_school_summary_df["% Overall Passing"].map("{:.2f}%".format)

per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


### Recalculate the high- and low-performing schools

In [21]:
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)
# Assign district summary df the new column order.

top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Total Budget
Cabrera High School,Charter,"1,858",1081356,582.0,83.1,84.0,94.13%,97.04%,91.33%,"$1,081,356.00"
Griffin High School,Charter,"1,468",917500,625.0,83.4,83.8,93.39%,97.14%,90.60%,"$917,500.00"
Wilson High School,Charter,"2,283",1319574,578.0,83.3,84.0,93.87%,96.54%,90.58%,"$1,319,574.00"
Pena High School,Charter,962,585858,609.0,83.8,84.0,94.59%,95.95%,90.54%,"$585,858.00"
Wright High School,Charter,"1,800",1049400,583.0,83.7,84.0,93.33%,96.61%,90.33%,"$1,049,400.00"


#### Effects on Thomas High School's Performance relative to other schools

Previously, Thomas High School was second in terms of all of the tops school. It had an average math score of 83.42%, an average reading score of 83.85%, and with % in Passing math and reading in 93.27% and 97.31% respectively. Overall, it used to have 90.95% passing. With the 9th grade scores eliminated, it is no longer a top-five school.

### Recalculate the scores by grade, scores by school spending, scores by school size, and scores by school type.

#### Effects of Ninth-grade Scores:  Math and Reading Scores by Grade

In [27]:
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

ninth_grader_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_grader_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_grader_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_grader_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

ninth_grader_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_grader_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_grader_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_grader_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

math_scores_by_grade = pd.DataFrame({
               "9th Grade Math Scores": ninth_grader_math_scores,
               "10th Grade Math Scores": tenth_grader_math_scores,
               "11th Grade Math Scores": eleventh_grader_math_scores,
               "12th Grade Math Scores": twelfth_grader_math_scores,               
                "9th Grade Reading Scores": ninth_grader_reading_scores,
               "10th Grade Reading Scores": tenth_grader_reading_scores,
               "11th Grade Reading Scores": eleventh_grader_reading_scores,
               "12th Grade Reading Scores": twelfth_grader_reading_scores})

math_scores_by_grade["9th Grade Math Scores"] = math_scores_by_grade["9th Grade Math Scores"].map("{:.1f}".format)
math_scores_by_grade["10th Grade Math Scores"] = math_scores_by_grade["10th Grade Math Scores"].map("{:.1f}".format)
math_scores_by_grade["11th Grade Math Scores"] = math_scores_by_grade["11th Grade Math Scores"].map("{:.1f}".format)
math_scores_by_grade["12th Grade Math Scores"] = math_scores_by_grade["12th Grade Math Scores"].map("{:.1f}".format)
math_scores_by_grade["9th Grade Reading Scores"] = math_scores_by_grade["9th Grade Reading Scores"].map("{:.1f}".format)
math_scores_by_grade["10th Grade Reading Scores"] = math_scores_by_grade["10th Grade Reading Scores"].map("{:.1f}".format)
math_scores_by_grade["11th Grade Reading Scores"] = math_scores_by_grade["11th Grade Reading Scores"].map("{:.1f}".format)
math_scores_by_grade["12th Grade Reading Scores"] = math_scores_by_grade["12th Grade Reading Scores"].map("{:.1f}".format)

math_scores_by_grade


,9th Grade Math Scores,10th Grade Math Scores,11th Grade Math Scores,12th Grade Math Scores,9th Grade Reading Scores,10th Grade Reading Scores,11th Grade Reading Scores,12th Grade Reading Scores
school_name,,,,,,,,
Bailey High School,77.1,77.0,77.5,76.5,81.3,80.9,80.9,80.9
Cabrera High School,83.1,83.2,82.8,83.3,83.7,84.3,83.8,84.3
Figueroa High School,76.4,76.5,76.9,77.2,81.2,81.4,80.6,81.4
Ford High School,77.4,77.7,76.9,76.2,80.6,81.3,80.4,80.7
Griffin High School,82.0,84.2,83.8,83.4,83.4,83.7,84.3,84.0
Hernandez High School,77.4,77.3,77.1,77.2,80.9,80.7,81.4,80.9
Holden High School,83.8,83.4,85.0,82.9,83.7,83.3,83.8,84.7
Huang High School,77.0,75.9,76.4,77.2,81.3,81.5,81.4,80.3
Johnson High School,77.2,76.7,77.5,76.9,81.3,80.8,80.6,81.2


Since we deleted data for THS's 9th grade reading and math scores, it shows up as NaN. However, this does not affect other data in this composite viewer across all schools for each grade.

#### Effects of Ninth-grade Scores: Scores by School Spending

In [28]:
per_school_capita.describe()
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

spending_summary_df = pd.DataFrame({
    "Average Math Score" : spending_math_scores,
    "Average Reading Score" : spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing": overall_passing_spending})

# Formatting
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)
spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,67,77,56
$645-675,77.0,81.0,66,81,54


With the 9th grade scores of THS eliminated, the percentages within spending range 630-644 shifted the most. With the average math score, percentage of students passing math and percentage of students passing reading lowered, it resulted with a lower overall passing percentage for the spending 630-644 spending range.

#### Effects of Ninth-grade Scores: Scores School Size

In [31]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})

size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)
size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.2f}%".format)
size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.2f}%".format)
size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.2f}%".format)

size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,93.55%,96.10%,89.88%
Medium (1000-2000),83.4,83.9,88.33%,91.26%,85.45%
Large (2000-5000),77.7,81.3,69.96%,82.77%,58.29%


Previously, 

With the 9th grade scores of THS eliminated, it seems that it has dramatically lowered the medium school size overall passing percentage, from 90.62% to 85.45%. This comes with the reduction of av

83.374684	83.864438	93.599695	96.790680	90.621535

#### Effects of Ninth-grade Scores: Socres by School Type

In [33]:
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

type_summary_df = pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})

type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)
type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)
type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.2f}%".format)
type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.2f}%".format)
type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.2f}%".format)

type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,90.33%,93.13%,87.20%
District,77.0,81.0,66.55%,80.80%,53.67%


With the 9th grade scores of THS eliminated, the average math score, % of students passing math, % of students passing, and overal passing percentage all droped for charter school type. With one exception, it seems the average reading score for remained the same. 